In [ ]:
import os
import whisper

# 加载Whisper模型
model = whisper.load_model("large-v3")

def transcribe_directory(directory_path):
    # 遍历文件夹内所有的.flac文件
    for filename in os.listdir(directory_path):
        if filename.endswith(".flac"):
            # 构建完整的文件路径
            file_path = os.path.join(directory_path, filename)
            
            # 调用Whisper模型进行语音转写
            result = model.transcribe(file_path, language="cantonese")  
            print(f"Transcription of {filename}: {result['text']}")


directory_path = "path/to/your/flac/files"
transcribe_directory(directory_path)
